In [1]:
import numpy as np
import pandas as pd
from scipy.stats import *

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mohitgabani1/Statistics/main/car_interest_rate.csv')

In [3]:
df

,atlanta,chicago,houston,memphis,newyork,philadelphia
0,13.75,14.25,14.00,15.00,14.50,13.50
1,13.75,13.00,14.00,14.00,14.00,12.25
2,13.50,12.75,13.51,13.75,14.00,12.25
3,13.50,12.50,13.50,13.59,13.90,12.00
4,13.00,12.50,13.50,13.25,13.75,12.00
5,13.00,12.40,13.25,12.97,13.25,12.00
6,13.00,12.30,13.00,12.50,13.00,12.00
7,12.75,11.90,12.50,12.25,12.50,11.90
8,12.50,11.90,12.50,11.89,12.45,11.90


# One way ANOVA:
#### one independent factor influence the sample data

### Manual Calculation:

In [4]:
# Null Hypothesis: mean interest rates are same among the cities
# alternate Hypothesis: mean interest rates are not same among cities

In [5]:
# step 1:
# grand total
T = np.sum(df.values)

# total datapoint in all the samples
N = df.size

# correction factor
correction_factor = T**2/N

print(correction_factor)

9135.362399999998


In [6]:
# step 2: between sample

# sum of square between sample:
ssb = np.sum(np.square(df.sum(axis=0))/len(df))-correction_factor

# degree of freedom: c - 1 -- c = no of columns
dfb = df.shape[1]-1

print({"ssb:": ssb}, {"dfb":dfb})

{'ssb:': 10.94566666666833} {'dfb': 5}


In [7]:
# step 3: total

# total sum of square: square of each datapoint
sst = np.sum(df.apply(np.square).sum(axis=0)) - correction_factor

# degree of freedom: N-1
dft = N-1

print({"sst:": sst}, {"dft":dft})

{'sst:': 32.70380000000296} {'dft': 53}


In [8]:
# step 4: within sample

# sum of square within sample
sse = sst-ssb

# degree of freedom: dft - dfb
dfe = dft-dfb

print({"sse:": sse}, {"dfe":dfe})

{'sse:': 21.758133333334627} {'dfe': 48}


In [9]:
# step 5

# mean sum of square between sample(mssb)
mssb = ssb/dfb

# mean sum of square within sample(msse)
msse = sse/dfe

print({'mssb:':mssb}, {'msse:':msse})

{'mssb:': 2.189133333333666} {'msse:': 0.45329444444447137}


In [10]:
# f statistics

F = mssb/msse

print(f"f statistics: {F}")

f statistics: 4.829384873702848


In [11]:
# critical value

critical_value = f.ppf(q=0.95, dfn=dfb, dfd=dfe)

print(f"critical value: {critical_value}")

critical value: 2.408514119499335


In [12]:
# p-value

p_value = 1-f.cdf(F, dfn=dfb, dfd=dfe)

print(f"p value: {p_value}")

p value: 0.0011745514145032887


In [13]:
# based on critical value
if F > critical_value:
    print('Reject Null Hypothesis')
else:
    print('Accept Null Hypothesis')
    
# based on p value
if p_value>0.05:
    print("Accept Null Hypothesis")
else:
    print('Reject Null Hypothesis')

Reject Null Hypothesis
Reject Null Hypothesis


### from scipy library:

In [14]:
f_oneway(*df.values.T)

F_onewayResult(statistic=4.8293848737024, pvalue=0.001174551414504048)

In [15]:
df.values.T

array([[13.75, 13.75, 13.5 , 13.5 , 13.  , 13.  , 13.  , 12.75, 12.5 ],
       [14.25, 13.  , 12.75, 12.5 , 12.5 , 12.4 , 12.3 , 11.9 , 11.9 ],
       [14.  , 14.  , 13.51, 13.5 , 13.5 , 13.25, 13.  , 12.5 , 12.5 ],
       [15.  , 14.  , 13.75, 13.59, 13.25, 12.97, 12.5 , 12.25, 11.89],
       [14.5 , 14.  , 14.  , 13.9 , 13.75, 13.25, 13.  , 12.5 , 12.45],
       [13.5 , 12.25, 12.25, 12.  , 12.  , 12.  , 12.  , 11.9 , 11.9 ]])

# Two way ANOVA:

In [16]:
# New drug types introduced to cure the high blood pressure.The dataset shows the reduction in blood pressure for particular
# drug category on male and female.
# we want to analyse that either drug type or gender type has made an impact on blood pressure reduction or not.

In [17]:
df = pd.DataFrame({'drug':['A','A','A','A','A','A','A','A','A','A','B','B','B','B','B','B','B','B','B','B'],
                  'gender':['male','male','male','male','male','female','female','female','female','female','male','male','male','male','male','female','female','female','female','female'],
                  'reduce_bp':[6,4,7,9,3,8,3,5,8,6,4,5,6,7,5,3,5,9,2,3]})
df.head()

,drug,gender,reduce_bp
0,A,male,6
1,A,male,4
2,A,male,7
3,A,male,9
4,A,male,3


In [18]:
# first hypothesis
## null hypothesis: there is no significant effect of drug type on blood pressure reduction
## alternate hypothesis: there is significant effect of drug type on blood pressure reduction

# second hypothesis
## null hypothesis: there is no significant effect of gender on blood pressure reduction
## alternate hypothesis: there is significant effect of gender on blood pressure reduction

## Manual Calculations:

In [19]:
categorical_column = ['drug','gender']
numeric_column = ['reduce_bp']

In [20]:
n = df.groupby(categorical_column).size().reset_index().iloc[0,-1]
print(f"number of datapoints in each category: {n}")

number of datapoints in each category: 5


In [21]:
crosstab = pd.crosstab(index=df[categorical_column[0]], columns=df[categorical_column[1]],\
                       values=df[numeric_column], aggfunc='mean', margins=True)
crosstab

gender,female,male,All
drug,,,
A,6.0,5.8,5.9
B,4.4,5.4,4.9
All,5.2,5.6,5.4


In [22]:
p = df[categorical_column[1]].nunique()
q = df[categorical_column[0]].nunique()
print(f"number of categories in group A = {p}")
print(f"number of categories in group B = {q}")

number of categories in group A = 2
number of categories in group B = 2


In [23]:
column_mean = crosstab.iloc[-1,0:-1].values
row_mean = crosstab.iloc[0:-1,-1].values
mean = crosstab.iloc[-1,-1] # overall mean

In [24]:
# sum of square between sample:
ssbw = n*((crosstab.iloc[0:-1,0:-1]-mean)**2).values.sum()

# degree of freedom: dfb = (c-1)*(r-1)-1
dfbw = p*q-1

print({'ssbw:':ssbw}, {'dfbw:':dfbw})

{'ssbw:': 7.599999999999996} {'dfbw:': 3}


In [25]:
# sum of square for category A: column
ssa = n*q*np.sum((column_mean-mean)**2)

# degree of freedom: dfc = c - 1
dfa = p-1

print({'ssa:':ssa}, {'dfa:':dfa})

{'ssa:': 0.7999999999999979} {'dfa:': 1}


In [26]:
# sum of square for category B: row
ssb = n*p*np.sum((row_mean-mean)**2)

# degree of freedom: dfr = r - 1
dfb = q-1

print({'ssb:':ssb}, {'dfb:':dfb})

{'ssb:': 5.0} {'dfb:': 1}


In [27]:
# sum square within sample: interaction
ssab = ssbw - ssa - ssb

# degree of freedom: dfe = (c-1)*(r-1) -- c & r from crosstab
dfab = (p-1)*(q-1)

print({'ssab:':ssab}, {'dfab:':dfab})

{'ssab:': 1.799999999999998} {'dfab:': 1}


In [28]:
# sum of square total:
sst = ((df[numeric_column]-mean)**2).values.sum()

# degree of freedom: dfb = (c-1)*(r-1)-1 -- c & r from crosstab
dft = n*p*q-1

print({'sst:':sst}, {'dft:':dft})

{'sst:': 84.80000000000001} {'dft:': 19}


In [29]:
# sum of square error:
cat_A = df[categorical_column[1]].unique()
cat_B = df[categorical_column[0]].unique()
print(f"Column categories: {cat_A}")
print(f"Row categories: {cat_B}")

Column categories: ['male' 'female']
Row categories: ['A' 'B']


In [30]:
sse = 0
for i in cat_A:
    for j in cat_B:
        data = df[(df[categorical_column[1]]==i)&(df[categorical_column[0]]==j)]
        data_mean = data[numeric_column].mean()
        sse+= ((data[numeric_column]-data_mean)**2).values.sum()
        
# degree of freedom: dfe
dfe = (n-1)*p*q

print({'sse:':sse}, {'dfe:':dfe})

{'sse:': 77.2} {'dfe:': 16}


In [31]:
# mean sum of square for category A: column
mssa = ssa/dfa

# mean sum of square for category B: row
mssb = ssb/dfb

# mean sum of square interaction
mssab = ssab/dfab

# mean sum of square error
msse =sse/dfe

print({'mssa:':mssa}, {'mssb:':mssb},{'mssab:':mssab}, {'msse:':msse})

{'mssa:': 0.7999999999999979} {'mssb:': 5.0} {'mssab:': 1.799999999999998} {'msse:': 4.825}


In [32]:
F_A = mssa/msse
F_B = mssb/msse
F_AB = mssab/msse

print({'F_A:':F_A}, {'F_B:':F_B},{'F_AB:':F_AB})

{'F_A:': 0.16580310880828972} {'F_B:': 1.0362694300518134} {'F_AB:': 0.37305699481865245}


In [33]:
p_a = 1-f.cdf(F_A, dfn=dfa, dfd=dfe)
p_b = 1-f.cdf(F_B, dfn=dfb, dfd=dfe)
p_ab = 1-f.cdf(F_AB, dfn=dfab, dfd=dfe)

print({'p_a:':p_a}, {'p_b:':p_b},{'p_ab:':p_ab})

{'p_a:': 0.6892646297178505} {'p_b:': 0.3238375983492294} {'p_ab:': 0.5499215848880679}


In [34]:
confidence_level = 0.95
significance_level = 1 - confidence_level

# checking first hypothesis:
if p_a<significance_level:
    print(f"{categorical_column[1]} has significant effect on blood pressure reduction")
else:
    print(f"{categorical_column[1]} has no significant effect on blood pressure reduction")
    
# checking other hypothesis:
if p_b<significance_level:
    print(f"{categorical_column[0]} has significant effect on blood pressure reduction")
else:
    print(f"{categorical_column[0]} has no significant effect on blood pressure reduction")
    
# checking hypothesis for interaction of both the categories
if p_ab<significance_level:
    print(f"{categorical_column[1]} + {categorical_column[0]} has significant effect on blood pressure reduction")
else:
    print(f"{categorical_column[1]} + {categorical_column[0]} has no significant effect on blood pressure reduction")

gender has no significant effect on blood pressure reduction
drug has no significant effect on blood pressure reduction
gender + drug has no significant effect on blood pressure reduction


## Using statsmodels: 

In [35]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [36]:
model = ols('reduce_bp~gender+drug+drug*gender', data=df).fit()

In [37]:
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
gender,1.0,0.8,0.800,0.165803,0.689265
drug,1.0,5.0,5.000,1.036269,0.323838
drug:gender,1.0,1.8,1.800,0.373057,0.549922
Residual,16.0,77.2,4.825,NaN,NaN
